In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Masking, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-05-02 19:51:36.360422: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-02 19:51:36.401844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 19:51:37.069684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [4]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [13]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition.variable_data_set import VariableDataSet
from swallowing_recognition.mlp import MLP
from swallowing_recognition.cnn import CNN

In [6]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_neck_movement_folder = directory_path / 'washino' / 'neck_movement'
train_shoulder_movement_folder = directory_path / 'washino' / 'shoulder_movement'
train_arm_movement_folder = directory_path / 'washino' / 'arm_movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   
train_mumble_folder = directory_path / 'washino' / 'mumble'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    
test_movement_folder = directory_path / 'shibata' / 'movement'    
test_hitting_folder = directory_path / 'shibata' / 'hitting'    

In [7]:
train_data = VariableDataSet(num_samples = 1400, scale = 222)
test_data = VariableDataSet(num_samples = 62, scale = 222)

In [8]:
train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0, signal_processing='fft')

In [9]:
train_data.folder_to_dataset(train_voice_folder, np.array(1), 700, signal_processing='fft')
train_data.folder_to_dataset(train_neck_movement_folder, np.array(1), 800, signal_processing='fft')
train_data.folder_to_dataset(train_shoulder_movement_folder, np.array(1), 900, signal_processing='fft')
train_data.folder_to_dataset(train_arm_movement_folder, np.array(1), 1000, signal_processing='fft')
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 1100, signal_processing='fft')
train_data.folder_to_dataset(train_cough_folder, np.array(1), 1200, signal_processing='fft')
train_data.folder_to_dataset(train_mumble_folder, np.array(1), 1300, signal_processing='fft')

857
[]
885
[]
908
[]
1065
[]
1076
[]
1103
[]
1283
[]


In [10]:
test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0, signal_processing='fft')
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14, signal_processing='fft')
test_data.folder_to_dataset(test_voice_folder, np.array(1), 28, signal_processing='fft')
test_data.folder_to_dataset(test_movement_folder, np.array(1), 42, signal_processing='fft')
test_data.folder_to_dataset(test_hitting_folder, np.array(1), 52, signal_processing='fft')

In [19]:
model = MLP(500, scale = 222)
model.training(train_data.data, train_data.labels, 80, 32)
model.evaluate(test_data.data, test_data.labels)
model.save('mlp_20240502_binary_ep80_bs32_v1.keras')

Epoch 1/80
40/40 [==============================] - 1s 10ms/step - loss: 0.5160 - accuracy: 0.8595 - val_loss: 0.4883 - val_accuracy: 0.7643
Epoch 2/80
40/40 [==============================] - 0s 6ms/step - loss: 0.2398 - accuracy: 0.9214 - val_loss: 1.0673 - val_accuracy: 0.7357
Epoch 3/80
40/40 [==============================] - 0s 6ms/step - loss: 0.1605 - accuracy: 0.9437 - val_loss: 1.2851 - val_accuracy: 0.7643
Epoch 4/80
40/40 [==============================] - 0s 5ms/step - loss: 0.1214 - accuracy: 0.9595 - val_loss: 1.7463 - val_accuracy: 0.7643
Epoch 5/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0943 - accuracy: 0.9690 - val_loss: 1.8525 - val_accuracy: 0.8000
Epoch 6/80
40/40 [==============================] - 0s 6ms/step - loss: 0.0800 - accuracy: 0.9762 - val_loss: 2.2319 - val_accuracy: 0.7786
Epoch 7/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0659 - accuracy: 0.9770 - val_loss: 2.5494 - val_accuracy: 0.7786
Epoch 8/80
40/40 [=

40/40 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 13.0523 - val_accuracy: 0.6786
Epoch 60/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 13.0667 - val_accuracy: 0.6786
Epoch 61/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.3853 - val_accuracy: 0.6786
Epoch 62/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 13.4217 - val_accuracy: 0.6786
Epoch 63/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.5212 - val_accuracy: 0.6786
Epoch 64/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 13.6355 - val_accuracy: 0.6786
Epoch 65/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.6263 - val_accuracy: 0.6786
Epoch 66/80
40/40 

In [15]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [16]:
model = CNN(scale = 222)
model.training(train_data.data, train_data.labels, 150, 32)
model.evaluate(test_data.data, test_data.labels)
model.save('cnn_20240502_binary_ep100_bs32_v1.keras')

Epoch 1/150
40/40 [==============================] - 3s 12ms/step - loss: 0.6151 - accuracy: 0.6587 - val_loss: 0.6822 - val_accuracy: 0.5000
Epoch 2/150
40/40 [==============================] - 0s 7ms/step - loss: 0.3627 - accuracy: 0.8595 - val_loss: 0.5493 - val_accuracy: 0.7357
Epoch 3/150
40/40 [==============================] - 0s 8ms/step - loss: 0.2849 - accuracy: 0.8929 - val_loss: 0.5081 - val_accuracy: 0.7643
Epoch 4/150
40/40 [==============================] - 0s 7ms/step - loss: 0.2498 - accuracy: 0.9103 - val_loss: 0.5660 - val_accuracy: 0.7429
Epoch 5/150
40/40 [==============================] - 0s 7ms/step - loss: 0.2221 - accuracy: 0.9230 - val_loss: 0.5448 - val_accuracy: 0.7714
Epoch 6/150
40/40 [==============================] - 0s 8ms/step - loss: 0.1982 - accuracy: 0.9270 - val_loss: 0.8569 - val_accuracy: 0.7071
Epoch 7/150
40/40 [==============================] - 0s 7ms/step - loss: 0.1741 - accuracy: 0.9381 - val_loss: 0.5082 - val_accuracy: 0.8357
Epoch 8/150


Epoch 59/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0149 - accuracy: 0.9960 - val_loss: 2.6235 - val_accuracy: 0.8000
Epoch 60/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0710 - accuracy: 0.9817 - val_loss: 2.0380 - val_accuracy: 0.8000
Epoch 61/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0285 - accuracy: 0.9913 - val_loss: 1.4750 - val_accuracy: 0.8143
Epoch 62/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0141 - accuracy: 0.9968 - val_loss: 1.4558 - val_accuracy: 0.8357
Epoch 63/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0149 - accuracy: 0.9944 - val_loss: 1.8914 - val_accuracy: 0.8214
Epoch 64/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0146 - accuracy: 0.9944 - val_loss: 1.5378 - val_accuracy: 0.8286
Epoch 65/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0072 - accuracy: 0.9984 - val_loss: 1.7846 - val_accuracy: 0.8143
Epoch 

40/40 [==============================] - 0s 8ms/step - loss: 9.9506e-04 - accuracy: 1.0000 - val_loss: 2.4227 - val_accuracy: 0.8429
Epoch 117/150
40/40 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 0.9992 - val_loss: 2.5024 - val_accuracy: 0.8357
Epoch 118/150
40/40 [==============================] - 0s 8ms/step - loss: 5.1495e-04 - accuracy: 1.0000 - val_loss: 2.4953 - val_accuracy: 0.8500
Epoch 119/150
40/40 [==============================] - 0s 8ms/step - loss: 3.8662e-04 - accuracy: 1.0000 - val_loss: 2.6212 - val_accuracy: 0.8500
Epoch 120/150
40/40 [==============================] - 0s 7ms/step - loss: 4.7442e-04 - accuracy: 1.0000 - val_loss: 2.6335 - val_accuracy: 0.8429
Epoch 121/150
40/40 [==============================] - 0s 7ms/step - loss: 3.4220e-04 - accuracy: 1.0000 - val_loss: 2.6059 - val_accuracy: 0.8429
Epoch 122/150
40/40 [==============================] - 0s 7ms/step - loss: 2.4738e-04 - accuracy: 1.0000 - val_loss: 2.6347 - val_accura

In [ ]:
model.evaluate(test_data.data, test_data.labels)